In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/s4_happy/happy_s4


In [2]:
from happy_s4.data.lit_data import HFLitDataArgs, HFLitDataModule

/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from happy_s4.model.lit_model import LitS4Args, LitS4
from happy_s4.model.s4_wrap import S4_GO_BRR_ARGS

CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%


In [4]:
lit_args = HFLitDataArgs(min_freq=1)

In [5]:
dm = HFLitDataModule(lit_args)

In [6]:
dm.setup()

Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f52da7787b76a170.arrow
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████████████████████████████████████████| 408/408 [00:00<00:00, 5686.76ex/s]


In [7]:
vocab_size = len(dm.tokenizer.vocab.get_itos())
pad_token_id = dm.tokenizer.pad_token_id


In [8]:
s4_args = S4_GO_BRR_ARGS(pad_token_id=pad_token_id, vocab_size=vocab_size, d_model=256, l_max=2000, channels=1,hurwitz=True, transposed=False)

In [9]:
lit_s4_args = LitS4Args(s4_args=s4_args)

In [10]:
lit_s4 = LitS4(lit_s4_args)

In [11]:
dl = dm.val_dataloader()

In [12]:
from pytorch_lightning import Trainer

In [13]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

In [14]:
es = EarlyStopping(monitor="val_loss")
mckpt = ModelCheckpoint(dirpath="outputs/mrpc/", filename="checkpoint_{epoch:02d}-{val_loss:03.0f}", monitor='val_loss', )

In [16]:
trainer = Trainer(callbacks=[es, mckpt], gpus=[1], max_epochs=1000)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model=lit_s4, datamodule=dm)

Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 774.71it/s]
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f52da7787b76a170.arrow
Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-11edd22d6dc380ff.arrow
/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint dire

Sanity Checking DataLoader 0:   0%|                                                               | 0/2 [00:00<?, ?it/s]

/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/mnt/data3/shared_envs/s4/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  90%|█████████████████████████████████████████████▊     | 115/128 [00:02<00:00, 43.32it/s, loss=0.997, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 1:  90%|█████████████████████████████████████████████▊     | 115/128 [00:05<00:00, 20.47it/s, loss=0.633, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 2:  90%|██████████████████████████████████████████████▋     | 115/128 [00:08<00:00, 13.43it/s, loss=0.57, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 3:  90%|█████████████████████████████████████████████▊     | 115/128 [00:11<00:01, 10.02it/s, loss=0.514, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 4:  90%|█████████████████████████████████████████████▊     | 115/128 [00:14<00:01,  8.01it/s, loss=0.442, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 5:  90%|█████████████████████████████████████████████▊     | 115/128 [00:17<00:01,  6.69it/s, loss=0.352, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 6:  90%|█████████████████████████████████████████████▊     | 115/128 [00:20<00:02,

In [81]:
lit_s4 = lit_s4.load_from_checkpoint("outputs/mrpc/checkpoint_epoch=03-val_loss=001.ckpt")

In [82]:
from happy_s4.data.data import WordTokenizer, shape_dataset
from happy_s4.data.collators import BatchCollators

In [83]:
import datasets
from functools import partial

In [84]:
from torch.utils.data import DataLoader

In [85]:
data_test = datasets.load_dataset(
    "glue",
    "mrpc",
    split="test",
)
data_test = data_test.map(
    partial(shape_dataset, word_tokenizer=dm.tokenizer)
)


Reusing dataset glue (/mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /mnt/data1/hf_dataset_cache/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-881645435afb9899.arrow


In [86]:
data_test = data_test.remove_columns(['sentence1', 'sentence2', 'idx'])

In [87]:
dl = DataLoader(
    dataset=data_test,
    collate_fn=BatchCollators(dm.tokenizer.pad_token_id),
    batch_size=dm.args.batch_size,
    shuffle=False
)

In [88]:
import torch

In [89]:
preds = []
gt = []

In [90]:
from tqdm import tqdm

In [91]:
for batch in tqdm(dl):
    with torch.no_grad(): 
        _, logits = lit_s4(input_ids=batch['input_ids'], labels=batch['label'])
        label = logits.argmax(dim=1).numpy().tolist()
        gt.extend(batch['label'].numpy().tolist())
        preds.extend(label)


100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 71.97it/s]


In [92]:
from sklearn.metrics import accuracy_score

In [93]:
accuracy_score(gt, preds)

0.6608695652173913